In [12]:
# %pip install seaborn


In [13]:
# %pip install numpy 
# %pip install matplotlib 
# %pip install pandas

In [14]:
# %pip install keras

In [15]:
# reading the files from the directory 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 

directory = r'D:\image processing\dataset'

for dirname, _, filenames in os.walk(directory):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# importing the needed libraries 

In [16]:
# %pip install cv2

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os

ModuleNotFoundError: No module named 'cv2'